In [ ]:
import sys
from pathlib import Path

# Add root folder to Python path (to import modules)
notebook_dir = Path().absolute()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

In [ ]:
import pandas as pd
from src.portfolios.portfolio_new import Portfolio
from src.portfolios.FF5FM_Mom import FF5FM_Mom

In [ ]:
results_df = pd.read_csv('/home/ge65cuw/thesis/saved/final_results/results_df.csv', index_col=0).dropna()
worst_net_id = results_df.loc[results_df.trial_id == results_df.trial_id.min()]['trial_id'].values[0]

In [ ]:
base_path = '/home/ge65cuw/thesis/saved/final_results/'
best_net_path = base_path + 'ensembleNN3.csv'
worst_net_path = base_path + 'predicted_returns/' + str(worst_net_id) + '_predicted_returns.csv'
benchmark_nn4_path = base_path + 'ensembleGu.csv'
linear_path = base_path + 'linear/predicted_ret.csv'
# market_returns = ''

In [ ]:
df = pd.read_csv(best_net_path, index_col=0)
pbest = Portfolio(df)

In [ ]:
portfolio = pbest
FFMom = FF5FM_Mom().returns
columns_to_keep = [portfolio.returns.columns[0]]
columns_to_keep.extend(portfolio.returns.columns[-2:])
df = portfolio.returns[columns_to_keep]
df = df.merge(FFMom, on=['yyyymm'], how='left')


# On long-short returns        
# X = df.iloc[:, 3:]

# Column 1 is long returns on max quantile, 
# Column 2 is long-short returns
# if method == 'long-short':
#     y = df.iloc[:,2]
# elif method == 'long':
#     y = df.iloc[:,1]

X = sm.add_constant(X)
lm = sm.OLS(y, X).fit()

print('Params')
print(lm.params)
print('tValues')
print(lm.tvalues)

alpha = lm.params[0]
t_value_alpha = lm.tvalues[0]
regression_rsquared = lm.rsquared
print('Linear Regression summary:')
print(lm.summary())


year_min = portfolio.returns['yyyymm'].min()//100
month_min = portfolio.returns['yyyymm'].min()%100

year_max = portfolio.returns['yyyymm'].max()//100
month_max = portfolio.returns['yyyymm'].max()%100

n_months =  ((year_max - year_min) + (month_max-month_min+1)/12)*12 
information_ratio_regression = t_value_alpha / np.sqrt(n_months)
print(f'Informatio Ratio based on FF5FM+Mom regression: {information_ratio_regression:.2f}')